# 获取各个节点的POI信息

In [1]:
import pickle
node_dict_amap={}
road_edges={}
with open('../data/map_nodes_amap.pkl','rb') as f:
    node_dict_amap=pickle.load(f)
with open('../data/road_edges.pkl','rb') as f:
    road_edges=pickle.load(f)

In [2]:
import requests
import json
import threading
import queue
from fake_useragent import UserAgent
def get_POI(location):
    server_host='http://106.11.43.113/v5/place/around'
    parameters={
        'key':'053d8830d4ed8bfc7fac9a3a5d8b10cf',
        'types':'050000|060000|070000|080000|090000|100000|110000|120000|130000|140000|150000|160000|170000',
        'location':str(location[0])[0:10]+','+str(location[1])[0:9],
        'radius':'200',
        'sortrule':'weight',
        'region':'上海市'
    }
    response=requests.get(url=server_host,
                         headers={'User-Agent':UserAgent(path='fakeuseragent.json').random},
                         params=parameters)
    response.encoding=response.apparent_encoding
    return response.text

q = queue.Queue()
thread_list = []

def download_task(node,location):
    res=json.loads(get_POI(location))
    if res['status']=="1" and res['info']=='OK':
        data=[{'name':poi['name'],
              'location':poi['location'],
              'type':poi['type'],
              'typecode':poi['typecode']} for poi in res['pois']]
        q.put((node,data))
    
node_list=[]
for road,edges in road_edges.items():
    node_list+=list(set([edge[0] for edge in edges]+[edge[1] for edge in edges]))
node_list=list(set(node_list))

node_dict_amap_POIs={}
while True:
    node_remain_list=[node for node in node_list if node not in node_dict_amap_POIs]
    if len(node_remain_list)==0:
        break
    
    for node in node_remain_list:
        location=node_dict_amap[node]
        args=(node,location)
        thread = threading.Thread(target=download_task, args=args)
        thread_list.append(thread)
        thread.start()

    for thread in thread_list:
        thread.join()
    thread_list.clear()


    while not q.empty():
        data=q.get()
        node_dict_amap_POIs[data[0]]=data[1]

# 保存数据

In [3]:
import pickle
with open('../data/POI.pkl','wb') as f:
    pickle.dump(node_dict_amap_POIs,f)

In [4]:
import pickle
with open('../data/POI.pkl','rb') as f:
    node_dict_amap_POIs=pickle.load(f)
print((len(node_dict_amap_POIs)))
print(node_dict_amap_POIs['601712444'])

4499
[{'name': '大眼面馆(总店)', 'location': '121.474306,31.206484', 'type': '餐饮服务;中餐厅;中餐厅', 'typecode': '050100'}, {'name': '上海启派贸易有限公司', 'location': '121.474414,31.206447', 'type': '公司企业;公司;公司', 'typecode': '170200'}, {'name': '上海药房(永春店)', 'location': '121.474268,31.206598', 'type': '医疗保健服务;医药保健销售店;药房', 'typecode': '090601'}, {'name': '黄家菜饭(丽园店)', 'location': '121.474528,31.206443', 'type': '餐饮服务;中餐厅;中餐厅', 'typecode': '050100'}, {'name': '爱心献血屋(丽园路店)', 'location': '121.474511,31.206455', 'type': '医疗保健服务;医疗保健服务场所;医疗保健服务场所', 'typecode': '090000'}, {'name': '大眼家常菜(上海鲁班路店)', 'location': '121.474271,31.206579', 'type': '餐饮服务;中餐厅;中餐厅', 'typecode': '050100'}, {'name': '西北牛肉面', 'location': '121.474247,31.206643', 'type': '餐饮服务;快餐厅;快餐厅', 'typecode': '050300'}, {'name': '老五超市', 'location': '121.474497,31.206436', 'type': '购物服务;便民商店/便利店;便民商店/便利店', 'typecode': '060200'}, {'name': '西北特色各种面饭(鲁班路老店)', 'location': '121.474249,31.206642', 'type': '餐饮服务;中餐厅;清真菜馆', 'typecode': '050121'}, {'name': '均均烟酒商行', '